Follows https://github.com/fastai/courses/blob/master/deeplearning2/densenet-keras.ipynb

In [41]:
import os

import matplotlib.pyplot as plt 
import scipy.misc
import numpy as np

import keras
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, Cropping2D
from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation, merge, AveragePooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.utils.layer_utils import print_summary
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras_tqdm import TQDMNotebookCallback
from keras.regularizers import l2

from keras import __version__ as kv
from scipy import __version__ as sv
from matplotlib import __version__ as mv
from numpy import __version__ as nv

print kv
print sv
print mv
print nv

2.0.5
0.19.1
2.0.0
1.13.0


In [2]:
# Local files
import utils
reload(utils)
import utils2
reload(utils2)

<module 'utils2' from 'utils2.pyc'>

# Data

### Config

In [3]:
trained_model_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/trained_models/3.0-as-densenet_from_scratch/'

In [4]:
train_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/train/50x50/'
valid_dir = '/home/ubuntu/data/sar/experiment_crops_20170815/validate/50x50/'

In [5]:
input_shape = (50, 50, 1)
num_classes = 3

### Convert tif2png

Done in 1.1-mph_as_kd-baseline-simple_CNN_from_scratch.ipynb

### Create Generators

In [47]:
# train_datagen = ImageDataGenerator()#rescale=1./255)

# test_datagen = ImageDataGenerator()#rescale=1./255)

# train_generator = train_datagen.flow_from_directory(
#         train_dir,
#         target_size=(50, 50),
#         batch_size=20,
#         class_mode='categorical', 
#         color_mode='grayscale')

# validation_generator = test_datagen.flow_from_directory(
#         valid_dir, 
#         target_size=(50, 50),
#         shuffle=False,
#         batch_size=10,
#         class_mode='categorical',
#         color_mode='grayscale')

Found 2000 images belonging to 3 classes.
Found 500 images belonging to 3 classes.


### Find and remove bad sized images

Let's create numpy array to find the bad sizes.

In [90]:
train_crops = []
train_labels = []
train_filenames = []

crops, labels, filenames = utils2.dir2list_of_numpy(train_dir+"/oil_and_gas_infrastructure/",
                                             [1,0,0])
train_crops.extend(crops)
train_labels.extend(labels)
train_filenames.extend(filenames)

crops, labels, filenames = utils2.dir2list_of_numpy(train_dir+"/other/",
                                             [0,1,0])
train_crops.extend(crops)
train_labels.extend(labels)
train_filenames.extend(filenames)

crops, labels, filenames = utils2.dir2list_of_numpy(train_dir+"/turbine/",
                                             [0,0,1])
train_crops.extend(crops)
train_labels.extend(labels)
train_filenames.extend(filenames)

In [91]:
valid_crops = []
valid_labels = []
valid_filenames = []

crops, labels, filenames = utils2.dir2list_of_numpy(valid_dir+"/oil_and_gas_infrastructure/",
                                             [1,0,0])
valid_crops.extend(crops)
valid_labels.extend(labels)
valid_filenames.extend(filenames)

crops, labels, filenames = utils2.dir2list_of_numpy(valid_dir+"/other/",
                                             [0,1,0])
valid_crops.extend(crops)
valid_labels.extend(labels)
valid_filenames.extend(filenames)

crops, labels, filenames = utils2.dir2list_of_numpy(valid_dir+"/turbine/",
                                             [0,0,1])
valid_crops.extend(crops)
valid_labels.extend(labels)
valid_filenames.extend(filenames)

In [92]:
print "train_crops:", len(train_crops)
print "train_labels:", len(train_labels)
print "train_filenames:", len(train_filenames)

print "valid_crops:", len(valid_crops)
print "valid_labels:", len(valid_labels)
print "valid_filenames:", len(valid_filenames)

train_crops: 2000
train_labels: 2000
train_filenames: 2000
valid_crops: 500
valid_labels: 500
valid_filenames: 500


In [93]:
for idx, array in enumerate(train_crops):
    if array.shape != (50, 50):
        print idx, array.shape, train_filenames[idx], train_labels[idx]
        
        # Change name from .png to .png_BAD_SIZE to prevent flow_from_directory picking them up. 
        os.rename(train_dir+"/other/"+train_filenames[idx], train_dir+"/other/"+train_filenames[idx]+"_BAD_SIZE")

509 (50, 40) S1A_IW_GRDH_1SDV_20170806T174108_20170806T174133_017806_01DD7A_EF7A_subset3_0.png [0, 1, 0]
582 (50, 49) S1B_IW_GRDH_1SDV_20170615T061318_20170615T061343_006057_00AA34_F4E6_subset7_12.png [0, 1, 0]
692 (50, 32) S1B_IW_GRDH_1SDV_20170615T061318_20170615T061343_006057_00AA34_F4E6_subset8_24.png [0, 1, 0]
756 (50, 49) S1B_IW_GRDH_1SDV_20170615T061318_20170615T061343_006057_00AA34_F4E6_subset4_8.png [0, 1, 0]


In [44]:
for idx, array in enumerate(valid_crops):
    if array.shape != (50, 50):
        print idx, array.shape, valid_filenames[idx], valid_labels[idx]
        
        # Change name from .png to .png_BAD_SIZE to prevent flow_from_directory picking them up. 
        os.rename(valid_dir+"/other/"+valid_filenames[idx], valid_dir+"/other/"+valid_filenames[idx]+"_BAD_SIZE")

Let's recreate numpy array without the bad sized files.

In [7]:
train_crops = []
train_labels = []
train_filenames = []

crops, labels, filenames = utils2.dir2list_of_numpy(train_dir+"/oil_and_gas_infrastructure/",
                                             [1,0,0])
train_crops.extend(crops)
train_labels.extend(labels)
train_filenames.extend(filenames)

crops, labels, filenames = utils2.dir2list_of_numpy(train_dir+"/other/",
                                             [0,1,0])
train_crops.extend(crops)
train_labels.extend(labels)
train_filenames.extend(filenames)

crops, labels, filenames = utils2.dir2list_of_numpy(train_dir+"/turbine/",
                                             [0,0,1])
train_crops.extend(crops)
train_labels.extend(labels)
train_filenames.extend(filenames)

In [8]:
valid_crops = []
valid_labels = []
valid_filenames = []

crops, labels, filenames = utils2.dir2list_of_numpy(valid_dir+"/oil_and_gas_infrastructure/",
                                             [1,0,0])
valid_crops.extend(crops)
valid_labels.extend(labels)
valid_filenames.extend(filenames)

crops, labels, filenames = utils2.dir2list_of_numpy(valid_dir+"/other/",
                                             [0,1,0])
valid_crops.extend(crops)
valid_labels.extend(labels)
valid_filenames.extend(filenames)

crops, labels, filenames = utils2.dir2list_of_numpy(valid_dir+"/turbine/",
                                             [0,0,1])
valid_crops.extend(crops)
valid_labels.extend(labels)
valid_filenames.extend(filenames)

In [98]:
print "train_crops:", len(train_crops)
print "train_labels:", len(train_labels)
print "train_filenames:", len(train_filenames)

print "valid_crops:", len(valid_crops)
print "valid_labels:", len(valid_labels)
print "valid_filenames:", len(valid_filenames)

train_crops: 1996
train_labels: 1996
train_filenames: 1996
valid_crops: 499
valid_labels: 499
valid_filenames: 499


In [9]:
# Reshape for keras format
np_train_filenames = np.array(train_filenames)
np_train_crops = np.array(train_crops)
np_train_labels = np.array(train_labels)

np_valid_filenames = np.array(valid_filenames)
np_valid_crops = np.array(valid_crops)
np_valid_labels = np.array(valid_labels)

np_train_crops = np.expand_dims(np_train_crops, axis=3)
np_train_filenames = np.expand_dims(np_train_filenames, axis=1)

np_valid_crops = np.expand_dims(np_valid_crops, axis=3)
np_valid_filenames = np.expand_dims(np_valid_filenames, axis=1)

In [11]:
print np_train_filenames.shape
print np_train_crops.shape
print np_train_labels.shape

print np_valid_filenames.shape
print np_valid_crops.shape
print np_valid_labels.shape

(1996, 1)
(1996, 50, 50, 1)
(1996, 3)
(499, 1)
(499, 50, 50, 1)
(499, 3)


# Experiments

Let's make some helper functions for piecing together our network using Keras' Functional API.

In [81]:
def relu(x): return Activation('relu')(x)
def dropout(x, p): return Dropout(p)(x) if p else x
def bn(x): return BatchNormalization(mode=0, axis=-1)(x)
def relu_bn(x): return relu(bn(x))


def conv(x, nf, sz, wd, p):
    x = Conv2D(nf, sz, sz, init='he_uniform', border_mode='same', 
                          W_regularizer=l2(wd))(x)
    return dropout(x,p)

def conv_block(x, nf, bottleneck=False, p=None, wd=0):
    x = relu_bn(x)
    if bottleneck: x = relu_bn(conv(x, nf * 4, 1, wd, p))
    return conv(x, nf, 3, wd, p)

def dense_block(x, nb_layers, growth_rate, bottleneck=False, p=None, wd=0):
    if bottleneck: nb_layers //= 2
    for i in range(nb_layers):
        b = conv_block(x, growth_rate, bottleneck=bottleneck, p=p, wd=wd)
        x = merge([x,b], mode='concat', concat_axis=-1)
    return x

def transition_block(x, compression=1.0, p=None, wd=0):
    nf = int(x.get_shape().as_list()[-1] * compression)
    x = relu_bn(x)
    x = conv(x, nf, 1, wd, p)
    return AveragePooling2D((2, 2), strides=(2, 2))(x)


In [82]:
def create_dense_net(nb_classes, img_input, depth=40, nb_block=3, 
     growth_rate=12, nb_filter=16, bottleneck=False, compression=1.0, p=None, wd=0, activation='softmax'):
    """
    nb_classes: number of classes
    img_input: tuple of shape (channels, rows, columns) or (rows, columns, channels)
    depth: total number of layers
        Includes 4 extra non-block layers
            1 input conv, 3 output layers
    nb_block: number of dense blocks (generally = 3).
        NOTE: Layers / block are evenly allocated. Therefore nb_block must be a factor of (Depth - 4)
    growth_rate: number of filters to add per dense block
    nb_filter: initial number of filters
    bottleneck: add bottleneck blocks
    Compression: Filter compression factor in transition blocks.
    p: dropout rate
    wd: weight decay
    activation: Type of activation at the top layer. Can be one of 'softmax' or 'sigmoid'.
                Note that if sigmoid is used, classes must be 1.
                
    Returns: keras tensor with nb_layers of conv_block appended
    """
    
    assert activation == 'softmax' or activation == 'sigmoid'
    assert (depth - 4) % nb_block == 0
    nb_layers_per_block = int((depth - 4) / nb_block)
    nb_layers = [nb_layers_per_block] * nb_block

    x = conv(img_input, nb_filter, 3, wd, 0)
    for i,block in enumerate(nb_layers):
        x = dense_block(x, block, growth_rate, bottleneck=bottleneck, p=p, wd=wd)
        if i != len(nb_layers)-1:
            x = transition_block(x, compression=compression, p=p, wd=wd)

    x = relu_bn(x)
    x = GlobalAveragePooling2D()(x)
    return Dense(nb_classes, activation=activation, W_regularizer=l2(wd))(x)

Test it on the SAR data:

In [14]:
img_input = Input(shape=input_shape)

In [33]:
x = create_dense_net(num_classes, img_input, depth=100, nb_filter=16, compression=0.5, 
                     bottleneck=True, p=0.2, wd=1e-4)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UserWarning:

Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same", kernel_regularizer=<keras.reg..., kernel_initializer="he_uniform")`

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning:

Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=-1)`

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UserWarning:

Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (1, 1), padding="same", kernel_regularizer=<keras.reg..., kernel_initializer="he_uniform")`

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UserWarning:

Update your `Conv2D` call to the Keras 2 API: `Conv2D(12, (3, 3), padding="same", kernel_regularizer=<keras.reg..., kernel_initializer="he_uniform")`

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:21: UserWarning:

The `merge` function is deprecated and will be removed after 08/201

In [34]:
model = Model(img_input, x)
model.compile(loss='sparse_categorical_crossentropy', 
      optimizer=keras.optimizers.SGD(0.1, 0.9, nesterov=True), metrics=["accuracy"])

In [92]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 50, 50, 1)     0                                            
____________________________________________________________________________________________________
conv2d_595 (Conv2D)              (None, 50, 50, 16)    160         input_5[0][0]                    
____________________________________________________________________________________________________
batch_normalization_595 (BatchNo (None, 50, 50, 16)    64          conv2d_595[0][0]                 
____________________________________________________________________________________________________
activation_595 (Activation)      (None, 50, 50, 16)    0           batch_normalization_595[0][0]    
___________________________________________________________________________________________

In [36]:
parms = {'verbose': 2, 'callbacks': [TQDMNotebookCallback()]}

In [91]:
K.set_value(model.optimizer.lr, 0.1)
model.fit(np_train_crops, np_train_labels,
          batch_size=32,
          epochs=10,
          validation_data=(np_valid_crops, np_valid_labels),
          **parms)

ValueError: Error when checking target: expected dense_7 to have shape (None, 1) but got array with shape (1996, 3)

Hmmm... let's go back to basics and ensure it works on CIFAR with this version of keras.

## CIFAR check

In [45]:
from keras.datasets.cifar10 import load_batch

def load_data():
    path = 'data/cifar-10-batches-py'
    num_train_samples = 50000
    x_train = np.zeros((num_train_samples, 3, 32, 32), dtype='uint8')
    y_train = np.zeros((num_train_samples,), dtype='uint8')
    for i in range(1, 6):
        data, labels = load_batch(os.path.join(path, 'data_batch_' + str(i)))
        x_train[(i - 1) * 10000: i * 10000, :, :, :] = data
        y_train[(i - 1) * 10000: i * 10000] = labels
    x_test, y_test = load_batch(os.path.join(path, 'test_batch'))
    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))
    x_train = x_train.transpose(0, 2, 3, 1)
    x_test = x_test.transpose(0, 2, 3, 1)
    return (x_train, y_train), (x_test, y_test)

In [46]:
(x_train, y_train), (x_test, y_test) = load_data()

In [47]:
x_train.shape

(50000, 32, 32, 3)

Check code works on CIFAR (as per Jeremy Howards notebook)

In [48]:
img_input = Input(shape=(32,32,3))

In [49]:
x = create_dense_net(10, img_input, depth=100, nb_filter=16, compression=0.5, 
                     bottleneck=True, p=0.2, wd=1e-4)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UserWarning:

Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same", kernel_regularizer=<keras.reg..., kernel_initializer="he_uniform")`

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning:

Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=-1)`

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UserWarning:

Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (1, 1), padding="same", kernel_regularizer=<keras.reg..., kernel_initializer="he_uniform")`

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UserWarning:

Update your `Conv2D` call to the Keras 2 API: `Conv2D(12, (3, 3), padding="same", kernel_regularizer=<keras.reg..., kernel_initializer="he_uniform")`

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:21: UserWarning:

The `merge` function is deprecated and will be removed after 08/201

In [50]:
model = Model(img_input, x)
model.compile(loss='sparse_categorical_crossentropy', 
      optimizer=keras.optimizers.SGD(0.1, 0.9, nesterov=True), metrics=["accuracy"])

In [51]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv2d_397 (Conv2D)              (None, 32, 32, 16)    448         input_2[0][0]                    
____________________________________________________________________________________________________
batch_normalization_397 (BatchNo (None, 32, 32, 16)    64          conv2d_397[0][0]                 
____________________________________________________________________________________________________
activation_397 (Activation)      (None, 32, 32, 16)    0           batch_normalization_397[0][0]    
___________________________________________________________________________________________

In [52]:
parms = {'verbose': 2, 'callbacks': [TQDMNotebookCallback()]}

In [55]:
K.set_value(model.optimizer.lr, 0.1)
model.fit(x_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(x_test, y_test),
          **parms)

Train on 50000 samples, validate on 10000 samples


Epoch 1/10


KeyboardInterrupt: 

It worked (on CIFAR)! Let's compare the data structure:

In [74]:
print np_train_crops.shape
print x_train.shape

print np_train_labels.shape
print y_train.shape

(1996, 50, 50, 1)
(50000, 32, 32, 3)
(1996, 3)
(50000, 1)


Hmmm... CIFAR is suppose to have 10 classes:

In [76]:
y_train

array([[6],
       [9],
       [9],
       ..., 
       [9],
       [1],
       [1]], dtype=uint8)

Ah...they're not one hot encoded yet. Let's try no one-hot encoding on SAR:

### Change one-hot encode labels to integers

In [80]:
np_train_labels_integers = np.argmax(np_train_labels, axis=1)
np_valid_labels_integers = np.argmax(np_valid_labels, axis=1)

print np_train_labels_integers.shape
print np_valid_labels_integers.shape
print np_train_labels_integers
print np_valid_labels_integers

(1996,)
(499,)
[0 0 0 ..., 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2

In [83]:
img_input = Input(shape=input_shape)

In [84]:
x = create_dense_net(num_classes, img_input, depth=100, nb_filter=16, compression=0.5, 
                     bottleneck=True, p=0.2, wd=1e-4)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UserWarning:

Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same", kernel_regularizer=<keras.reg..., kernel_initializer="he_uniform")`

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning:

Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=-1)`

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UserWarning:

Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (1, 1), padding="same", kernel_regularizer=<keras.reg..., kernel_initializer="he_uniform")`

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UserWarning:

Update your `Conv2D` call to the Keras 2 API: `Conv2D(12, (3, 3), padding="same", kernel_regularizer=<keras.reg..., kernel_initializer="he_uniform")`

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:21: UserWarning:

The `merge` function is deprecated and will be removed after 08/201

In [85]:
model = Model(img_input, x)
model.compile(loss='sparse_categorical_crossentropy', 
      optimizer=keras.optimizers.SGD(0.1, 0.9, nesterov=True), metrics=["accuracy"])

In [86]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 50, 50, 1)     0                                            
____________________________________________________________________________________________________
conv2d_595 (Conv2D)              (None, 50, 50, 16)    160         input_5[0][0]                    
____________________________________________________________________________________________________
batch_normalization_595 (BatchNo (None, 50, 50, 16)    64          conv2d_595[0][0]                 
____________________________________________________________________________________________________
activation_595 (Activation)      (None, 50, 50, 16)    0           batch_normalization_595[0][0]    
___________________________________________________________________________________________

In [87]:
parms = {'verbose': 2, 'callbacks': [TQDMNotebookCallback()]}

In [89]:
K.set_value(model.optimizer.lr, 0.1)
model.fit(np_train_crops, np_train_labels_integers,
          batch_size=32,
          epochs=10,
          validation_data=(np_valid_crops, np_valid_labels_integers),
          **parms)

Train on 1996 samples, validate on 499 samples


Epoch 1/10
1660s - loss: 1.3725 - acc: 0.7004 - val_loss: 12.6994 - val_acc: 0.2505


Epoch 2/10


KeyboardInterrupt: 

Hmmm...slow to run.